In [1]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo


__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


def alexnet(pretrained=False, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = AlexNet(**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['alexnet']))
    return model

In [2]:
model = alexnet(True)

In [3]:
for param in model.classifier.parameters():
    print(type(param.data), param.size())
    i = 0
    with open("foo.txt", "w") as f:
        f.write(str(param.shape[0])+" "+str(param.shape[1])+"\n")
        for row in param:
            for val in row:
                f.write(str(float(val.data))+" ")
            f.write("\n")
            if i%100 == 0:
                print(i)
            i = i+1
    break

<class 'torch.Tensor'> torch.Size([4096, 9216])
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


**Generate mat.csv from bcm.cpp file as follows:**
```sh
$ g++ -O3 bcm.cpp
$ ./a.out foo.txt mat.csv 4
```

In [3]:
import numpy as np

matrix = np.genfromtxt('mat.csv', delimiter=' ')
print(matrix.size)


37748736


In [3]:
import torch
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(os.path.abspath('ILSVRC2012_img_val'), transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=10, shuffle=False, pin_memory=True)

In [4]:
import time

criterion = nn.CrossEntropyLoss()

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))

    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top1.avg

In [16]:
validate(val_loader, model, criterion)

/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


 * Prec@1 56.522 Prec@5 79.066


tensor(56.5220)

In [40]:
import numpy as np

matrix = np.genfromtxt('mat.csv', delimiter=' ')
print(matrix.size)

matrix = torch.Tensor(matrix)
model.state_dict()['classifier.1.weight'].data.copy_(matrix)
validate(val_loader, model, criterion)

/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


 * Prec@1 38.210 Prec@5 64.340


tensor(38.2100)

In [5]:
import numpy as np

matrix = np.genfromtxt('mat.csv', delimiter=' ')
print(matrix.size)

matrix = torch.Tensor(matrix)
model.state_dict()['classifier.1.weight'].data.copy_(matrix)
validate(val_loader, model, criterion)

37748736


/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/prateek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Test: [0/5000]	Time 1.325 (1.325)	Loss 2.8399 (2.8399)	Prec@1 70.000 (70.000)	Prec@5 90.000 (90.000)
Test: [100/5000]	Time 0.504 (0.495)	Loss 2.8699 (2.5159)	Prec@1 70.000 (70.990)	Prec@5 90.000 (87.822)
Test: [200/5000]	Time 0.446 (0.657)	Loss 3.0745 (2.8579)	Prec@1 30.000 (64.677)	Prec@5 90.000 (84.577)
Test: [300/5000]	Time 0.470 (0.601)	Loss 4.2991 (3.1244)	Prec@1 20.000 (59.568)	Prec@5 60.000 (81.661)
Test: [400/5000]	Time 0.480 (0.572)	Loss 1.8177 (3.1867)	Prec@1 100.000 (57.506)	Prec@5 100.000 (80.748)
Test: [500/5000]	Time 0.481 (0.555)	Loss 1.4061 (3.0267)	Prec@1 90.000 (60.898)	Prec@5 100.000 (82.655)
Test: [600/5000]	Time 0.484 (0.543)	Loss 3.4103 (3.0781)	Prec@1 70.000 (60.932)	Prec@5 70.000 (82.413)
Test: [700/5000]	Time 0.489 (0.544)	Loss 2.3944 (3.0504)	Prec@1 80.000 (61.583)	Prec@5 90.000 (82.810)
Test: [800/5000]	Time 0.492 (0.535)	Loss 3.2778 (3.0404)	Prec@1 60.000 (61.923)	Prec@5 80.000 (83.171)
Test: [900/5000]	Time 0.474 (0.530)	Loss 3.4836 (3.0881)	Prec@1 40.000 (

tensor(51.7100)